# **Auto Correct**

In this notebook, we are going to build an **auto-correct** (English Language) system that is very effective and useful.

### **Overview**

We use autocorrect every day on our cell phone and computer. In this notebook, we are going to explore what really goes on behind the scenes. Of course, the model we are going to implement is not identical to the one we use in our phone, but it is still quite good for a beginner (like me). 

- For example, if we type in the word **"I am lerningg"**, chances are very high that we meant to write **"learning"**. 

<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://angqpdfr.coursera-apps.org/notebooks/Week1/auto-correct.png' alt="alternate text" width="width" height="height" style="width:300px;height:250px;" /></div>

#### **Edit Distance**

We will implement models that correct words that are 1 and 2 edit distances away. 
- We say two words are n edit distance away from each other when we need n edits to change one word into another. 

An edit could consist of one of the following options: 

- Delete (remove a letter): *‘hat’ => ‘at, ha, ht’*
- Switch (swap 2 adjacent letters): *‘eta’ => ‘eat, tea,...’*
- Replace (change 1 letter to another): *‘jat’ => ‘hat, rat, cat, mat, ...’*
- Insert (add a letter): *‘te’ => ‘the, ten, ate, ...’*

We will use the above four methods to implement an **Auto-correct**. 
- To do so, we will need to compute probabilities that a certain word is correct given an input. 

This auto-correct, that we are going to see very soon,was first created by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) in 2007. 
- His [original article](https://norvig.com/spell-correct.html) may be a useful reference for better understanding.

The goal of our spell check model is to compute the following probability:

$$P(c|w) = \frac{P(w|c)\times P(c)}{P(w)}$$

The equation above is [Bayes Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem). 
- The equation says that the probability of a word being correct $P(c|w) $is equal to the probability of having a certain word $w$, given that it is correct $P(w|c)$, multiplied by the probability of being correct in general $P(C)$ divided by the probability of that word $w$ appearing $P(w)$ in general.
- To compute this equation, we will first import a data set and then create all the probabilities that we need using that data set. 

### **Data Preprocessing**

In [7]:
import re
from collections import Counter
import numpy as np
import pandas as pd

To implement this technique, we will use real-world data - **'shakespeare.txt'**.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
! cp '/content/drive/My Drive/Colab Notebooks/NLP-with-probabilistic-models/autocorrect/shakespeare.txt' '/content'

We will start with implementing a function `process_data` which:

1) Reads in a corpus (text file)

2) Changes everything to lowercase

3) Returns a list of words 

In [8]:
def process_data(file_name):
    """
    Input: 
        a file_name which is found in our current directory. 
    Output: 
        words: a list containing all the words in the corpus (text file we read) in lower case. 
    """
    words = []

    with open(file_name) as f:
        file_name_data = f.read()
    file_name_data=file_name_data.lower()
    words = re.findall('\w+',file_name_data)
    
    return words

The above function returns a `list` of words. We will convertthis list to a python `set`. This eliminates any duplicate entries.

In [9]:
word_l = process_data('shakespeare.txt')
vocab = set(word_l)  # this will be our vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


Next, we will implement a `get_count` function that returns a dictionary
- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus. 

For example, given the following sentence: **"I am happy because I am learning"**, our dictionary will return the following: 
<table style="width:20%">

  <tr>
    <td> <b>Key </b>  </td>
    <td> <b>Value </b> </td> 


  </tr>
  <tr>
    <td> I  </td>
    <td> 2</td> 
 
  </tr>
   
  <tr>
    <td>am</td>
    <td>2</td> 
  </tr>

  <tr>
    <td>happy</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>because</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>learning</td>
    <td>1</td> 
  </tr>
</table>  


In [10]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: the wordcount dictionary where key is the word and value is its frequency.
    '''
    
    word_count_dict = {}
    #Counter function in Python collections counts the frequencies of each key in a dict for us
    word_count_dict = Counter(word_l)

    return word_count_dict

In [11]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")

There are 6116 key values pairs
The count for the word 'thee' is 240


Next, given the dictionary of word counts, we will compute the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{M}$$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus,

$M$ is the total number of words in the corpus,

For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'** is:

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{7}$$


In [12]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: the wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: a dictionary where keys are the words, and the values are the probability that a word will occur. 
    '''
    probs = {}
    
    m = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m
    
    return probs

In [13]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


### **String Manipulations**

Now that we have computed $P(w_i)$ for all the words in the corpus, we will write a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words. To achieve this, we will implement four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 


In [14]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which we will generate all possible words 
              in the vocabulary which have 1 missing character.
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word.
    '''
    
    delete_l = []
    split_l = []
    
    for c in range(len(word)):
        split_l.append((word[:c],word[c:]))
    for a,b in split_l:
        delete_l.append(a+b[1:])          
    
    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [15]:
# testing the function
delete_word_l = delete_letter(word="cans",
                        verbose=True)

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')], 
delete_l = ['ans', 'cns', 'cas', 'can']


In [16]:
# another test of the function
print(f"Number of outputs of delete_letter('at') is {len(delete_letter('at'))}")

Number of outputs of delete_letter('at') is 2


In [17]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string.
     Output:
        switches: a list of all possible strings with one adjacent charater switched.
    '''
    def swap(c, i, j):
        c = list(c)
        c[i], c[j] = c[j], c[i]
        return ''.join(c)
    
    switch_l = []
    split_l = []
    len_word=len(word)
    
    for c in range(len_word):
        split_l.append((word[:c],word[c:]))
    switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]      
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [18]:
# testing the function
switch_word_l = switch_letter(word="eta",
                         verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [19]:
# another test of the function
print(f"Number of outputs of switch_letter('at') is {len(switch_letter('at'))}")

Number of outputs of switch_letter('at') is 1


In [20]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word.
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    '''

    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    for c in range(len(word)):
        split_l.append((word[0:c],word[c:]))
    replace_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
    replace_set=set(replace_l)    
    replace_set.remove(word)
    
    # turning the set back into a list and sorting it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [21]:
# testing the function
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [22]:
# another test of the function
print(f"Number of outputs of switch_letter('at') is {len(switch_letter('at'))}")

Number of outputs of switch_letter('at') is 1


In [27]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word.
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset.
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    for c in range(len(word)+1):
        split_l.append((word[0:c],word[c:]))
    insert_l = [ a + l + b for a,b in split_l for l in letters]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [28]:
# testing the function
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word = at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


### **Combining the edits**

We have implemented the required string manipulations. Now, we will create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

#### **Edit one letter**

The `edit_one_letter` function will return all the possible edits that are one edit away from a word. The edits consist of the replace, insert, delete, and optionally the switch operation. 

The 'switch' function  is a less common edit function, so its use will be selected by an "allow_switches" input argument.

Note that previously defined functions return *lists* while this function will return a *python set*. Utilizing a set eliminates any duplicate entries.

In [29]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible words that are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit.
    """
    
    edit_one_set = set()
        
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    
    return edit_one_set

In [30]:
# testing the function
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)

# turning this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


#### **Edit two letters**

Now we can generalize this to get two edits on a word. To do so, we will have to get all the possible edits on a single word, and then for each modified word, we will have to modify it again. 

The `edit_two_letters` function will returns a set of words that are two edits away. We will use `edit_two_letters` function to get a set of all the possible 2 edits on our word. We

 will then use those strings to get the most probable word we meant to type, aka our typing suggestion.

In [31]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word.
    Output:
        edit_two_set: a set of strings with all possible two edits.
    '''
    
    edit_two_set = set()
    
    edit_one = edit_one_letter(word,allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    
    return edit_two_set

In [32]:
# testing the function
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


### **Spelling suggestions**

Next, we will implement a function `get_corrections`, which returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word). 

There steps involved in this process are:

**Step 1:** Generate suggestions for a supplied word: We'll use the edit functions we have developed. The 'suggestion algorithm' will follow this logic: 
* If the word is in the vocabulary, suggest the word. 
* Otherwise, if there are suggestions from `edit_one_letter` that are in the vocabulary, use those. 
* Otherwise, if there are suggestions from `edit_two_letters` that are in the vocabulary, use those. 
* Otherwise, suggest the input word.*  
* The idea is that words generated from fewer edits are more likely than words with more edits.


Note: 
- Edits of one or two letters may 'restore' strings to either zero or one edit. This algorithm accounts for this by preferentially selecting lower distance edits first.

**Step 2**: Create a 'best_words' dictionary where the 'key' is a suggestion and the 'value' is the probability of that word in your vocabulary. If the word is not in the vocabulary, assign it a probability of 0.

**Step 3**: Select the n best suggestions. There may be fewer than n.

In [33]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions.
        probs: a dictionary that maps each word to its probability in the corpus.
        vocab: a set containing all the vocabulary.
        n: number of possible word corrections we want to be returned in the dictionary.
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    n_best = [[s,probs[s]] for s in list(reversed(suggestions))]
   
    if verbose: print("suggestions = ", suggestions)

    return n_best

In [35]:
# testing the function
my_word = 'dys' 

tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=False)
tmp_corrections.sort()
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

print(f"data type of corrections {type(tmp_corrections)}")

word 0: days, probability 0.000410
word 1: dye, probability 0.000019
data type of corrections <class 'list'>


In [40]:
# testing the function with another word
my_word = 'learnnig' 

tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=False)
tmp_corrections.sort()
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

word 0: learning, probability 0.000075


In this case, the function returned only single suggestion word even when we specified to return two most probable words. This means that, for the input word, there is only one probable english word that could be suggested.